In [1]:
import pandas as pd
import random
import numpy as np
import seaborn as sns
import math
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier,RidgeClassifier

In [2]:
X=pd.read_csv('x_train.csv',delimiter=";",header=None,names=['f'+str(i) for i in range (223)])
X_test=pd.read_csv('x_test.csv',delimiter=";",header=None,names=['f'+str(i) for i in range (223)])
y=pd.read_csv('y_train.csv',names=['class'])
y=y['class']

In [36]:
X_ftr=pd.read_csv('ftr_train.csv')
X_ftr.drop('Unnamed: 0',inplace=True,axis=1)

FileNotFoundError: File b'ftr_train.csv' does not exist

In [3]:
# Фуникция считает качество модели на кроссвалилации
# clf - модель, X,y - данные, ftr_set - пподмножество названий колонок, на которых считаем модель
# cv_val - количество холдов для кросс валидации
def CV_score(clf,X,y,ftr_set,cv_val):
    # Если подмоножество признаков пустое, то возвращаем 0 
    if len(ftr_set)==0:
        return 0
    # Иначе делаем кроссвалидацию
    scores = cross_val_score(clf, X[ftr_set],y,cv=cv_val)
    return scores.mean()
    #return random.random()

# Функция определяет  фичу, добавление которой наилучшим образом увеличивает качество 
# Фуникция берет случайную фичу из диапазона, добавляет её к стартовому набору фич и строит качество
# N -  максимальное количество итераций
# start_ftr - стартовый набор признаков
# start_cv - стартовое значение качества
def get_best_ftr(clf,X,y,start_ftr,start_cv,cv_val,N_params):
    print ("===START ftr lookup")
    print ("Start_ftr_set=",start_ftr)
    print ("Start_cv=     ",start_cv)
    i=0
    max_iter=N_params[0]
    start_ftr_N=N_params[1]
    end_ftr_N=N_params[2]
    used_ftr=start_ftr.copy()
    best_cv=start_cv
    best_ftr=''
    feature_names=list(X.columns)
    #while i<max_iter:
    for ftr_index in range(max_iter):
        # выбираем случайную фичу
        #ftr_index=random.randint(start_ftr_N,end_ftr_N)
        ftr_name=feature_names[ftr_index]
        # Проверяем, что её не использвоали
        if ftr_name in used_ftr:
            continue
        # Считаем качество на наборе фич с учетом выбранной
        ftr_set=start_ftr.copy()
        ftr_set.append(ftr_name)
        cv=CV_score(clf,X,y,ftr_set,cv_val)
        if cv>best_cv:
            best_cv=cv
            best_ftr=ftr_name
            print ("  ! ",end='')
        else:
            print ("    ",end='')
        print ("{:5} {:10}".format(i,ftr_name), " ",cv)
        used_ftr.append(ftr_name)
        i=i+1
    print ("===END ftr lookup ")
    return best_ftr,best_cv

# Фуникция производит отбор фич методом Sequential Feature Selection
def sfs(clf,X,y,start_ftr,cv_val,N_params):
    cur_ftr=start_ftr.copy()
    best_cv=CV_score(clf,X,y,start_ftr,cv_val)
    doAdd=True
    while doAdd:
        new_ftr,best_cv=get_best_ftr(clf,X,y,cur_ftr,best_cv,cv_val,N_params)
        if (new_ftr!=''):
            print (new_ftr+" added! New cv=",best_cv)
            cur_ftr.append(new_ftr)
        else: doAdd=False
    print ("Final_ftr_set=",cur_ftr, " final_cv=",best_cv)
    return cur_ftr,best_cv

In [9]:
#clf = RandomForestClassifier(max_features='auto',n_estimators=300,max_depth=100,random_state=42)
start_ftr=['f138', 'f96', 'f76', 'f11', 'f185', 'f115', 'f131', 'f83']
#Final_ftr_set= ['f138', 'f11', 'f50', 'f196']  final_cv= 0.496697603886
clf=SGDClassifier(random_state=42,alpha=0.001,loss='log',penalty='l1')
#start_ftr=[]
max_iter=223
start_ftr_N=0
end_ftr_N=223
N_params=(max_iter, start_ftr_N,end_ftr_N)
best_ftr,best_cv=sfs(clf,X,y,start_ftr,5,N_params)

===START ftr lookup
Start_ftr_set= ['f138', 'f96', 'f76', 'f11', 'f185', 'f115', 'f131', 'f83']
Start_cv=      0.494979310735
  !     0 f0           0.501569585072
        1 f1           0.47605209385
        2 f2           0.470621880365
  !     3 f3           0.501854084894
        4 f4           0.495259676065
        5 f5           0.464848503754
        6 f6           0.492116003781
        7 f7           0.470893248643
        8 f8           0.493547865168
        9 f9           0.485517122344
       10 f10          0.498415665953
       11 f12          0.474617328933
       12 f13          0.458003352115
       13 f14          0.466676022789
       14 f15          0.496127913833
       15 f16          0.500421391907
       16 f17          0.426688141122
       17 f18          0.481792966734
       18 f19          0.495838092105
       19 f20          0.49698299415
       20 f21          0.501282642207
       21 f22          0.500994466039
       22 f23          0.491833985843
  

In [6]:
Получится ли 156?

'f222f222'

# ADD-DEL

In [24]:
# Вычисляем качество
def get_cv_score(clf,cur_set):
    if len(cur_set)==0:
        return 0
    #rf=RandomForestClassifier(max_features='auto',n_estimators=300,max_depth=100,random_state=1)
    scores = cross_val_score(clf, X[cur_set],y,cv=5)
    return scores.mean()
    #return (np.mean(cross_val_score(rf, X[cur_set], y, cv=5)))
    

In [25]:
# Вспомогательная фуникця для ADD шага
# Определяем признак, добавление которого в набор приводит к лучшему изменению качества
def add_one_step(clf,ftr_set):
    cols=X.columns
    cv_df=pd.DataFrame(columns=['score'])
    # Последовательно перебирем все признаки, которые можем добавить
    print (   "currentFtr=",end=" ")
    for ftr in cols:
        #  и которые уже не входят в текущий набор признаков
        if ftr in ftr_set:
            continue
        # оцениваем качество для очередного признака
        cur_ftr_set=ftr_set.copy()
        cur_ftr_set.append(ftr)
        print (   ftr,end=" ")
        cv_df.loc[ftr]= get_cv_score(clf,cur_ftr_set)
    # Возвращаем признак на котором достигли лучшего качества и саму метрику
    return (cv_df.idxmax()[0], cv_df.max()[0])

# Функция делает полный ADD шаг
# Возвращает количество признаков, которые добавлены с увеличение качества
def add_step(clf,cur_set):
    print ("===Start ADD step")
    DoAdd=True
    cur_cv=get_cv_score(clf,cur_set)
    stps=0
    # Пытаемся добавлять признаки, пока растет качество
    while DoAdd:
        # Добавляем очередной признак и считаем качество
        add_ftr,add_cv=add_one_step(clf,cur_set)
        print ("Step=",stps," Curset=",cur_set,"  CurCV=",cur_cv)
        print ("  BestFtr=",add_ftr,"  BestCV=",add_cv)
        # Качество выросло?
        if add_cv>cur_cv:
            # Если да, то добавляем признак в итоговый набор признаков
            cur_set.append(add_ftr)
            cur_cv=add_cv
            stps=stps+1
            print ("  append ftr")
        # Если качество не растет, то заканчиваем ADD шаг
        else: DoAdd=False
    print ("END Add step. Curset=",cur_set,"  CurCV=",cur_cv) 
    return stps,cur_set

# Вспомогательная фуникця для DEL шага    
# Определяем признак, исключение которого из набор приводит к лучшему изменению качества
def del_one_step(clf,ftr_set):
    cv_df=pd.DataFrame(columns=['score'])
    # Последовательно перебирем все признаки из исходного множества
    for ftr in ftr_set:
        # исключаем признак из набора
        cur_ftr_set=ftr_set.copy()
        cur_ftr_set.remove(ftr)
        # считаем качество
        cv_df.loc[ftr]= get_cv_score(clf,cur_ftr_set)
    # Возвращаем признак на котором достигли лучшего качества и саму метрику
    return (cv_df.idxmax()[0], cv_df.max()[0])

# Функция делает полный DEL шаг
# Возвращает количество признаков, которые удалены с увеличение качества
def del_step(clf,cur_set):
    print ("===Start DEL step")
    DoDel=True
    cur_cv=get_cv_score(clf,cur_set)
    stps=0
    # Пытаемся удалять признаки, пока растет качество
    while DoDel:
        # Удаляем очередной признак и считаем качество
        del_ftr,del_cv=del_one_step(clf,cur_set)
        print ("Step=",stps," Curset=",cur_set,"  CurCV=",cur_cv)
        print ("  BestFtr=",del_ftr,"  BestCV=",del_cv)
        # Качество выросло?
        if del_cv>cur_cv:
            # Если да, то добавляем признак в итоговый набор признаков
            cur_set.remove(del_ftr)
            cur_cv=del_cv
            stps=stps+1
            print ("  delete ftr")
        # Если качество не растет, то заканчиваем ADD шаг
        else: DoDel=False
    print ("End DEL step. Curset=",cur_set,"  CurCV=",cur_cv) 
    return stps,cur_set

In [33]:
def add_del(clf,start_set):
    k=5
    while k>0:
        print ("=Start ADD-DEL Step")
        add_st,cur_set=add_step(clf,start_set)   
        print (">",add_st,cur_set)
        del_st,cur_set=del_step(clf,start_set)
        print (">",del_st,cur_set)
        k=del_st+add_st

In [35]:
cur_set= ['f138', 'f11', 'f37', 'f221', 'f182', 'f61', 'f64', 'f93']
clf=SGDClassifier(random_state=42,alpha=0.001,loss='log',penalty='l1')
add_del(clf,cur_set)

=Start ADD-DEL Step
===Start ADD step
currentFtr= f0 f1 f2 f3 f4 f5 f6 f7 f8 f9 f10 f12 f13 f14 f15 f16 f17 f18 f19 f20 f21 f22 f23 f24 f25 f26 f27 f28 f29 f30 f31 f32 f33 f34 f35 f36 f38 f39 f40 f41 f42 f43 f44 f45 f46 f47 f48 f49 f50 f51 f52 f53 f54 f55 f56 f57 f58 f59 f60 f62 f63 f65 f66 f67 f68 f69 f70 f71 f72 f73 f74 f75 f76 f77 f78 f79 f80 f81 f82 f83 f84 f85 f86 f87 f88 f89 f90 f91 f92 f94 f95 f96 f97 f98 f99 f100 f101 f102 f103 f104 f105 f106 f107 f108 f109 f110 f111 f112 f113 f114 f115 f116 f117 f118 f119 f120 f121 f122 f123 f124 f125 f126 f127 f128 f129 f130 f131 f132 f133 f134 f135 f136 f137 f139 f140 f141 f142 f143 f144 f145 f146 f147 f148 f149 f150 f151 f152 f153 f154 f155 f156 f157 f158 f159 f160 f161 f162 f163 f164 f165 f166 f167 f168 f169 f170 f171 f172 f173 f174 f175 f176 f177 f178 f179 f180 f181 f183 f184 f185 f186 f187 f188 f189 f190 f191 f192 f193 f194 f195 f196 f197 f198 f199 f200 f201 f202 f203 f204 f205 f206 f207 f208 f209 f210 f211 f212 f213 f214 f215 f216 f217 

# Curset= ['f138', 'f11', 'f37', 'f221', 'f182', 'f61', 'f64', 'f93']   CurCV= 0.518782142889